In [ ]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [ ]:
ngl_base = 'https://neuromancer-seung-import.appspot.com/' # this will point to the public bucket


In [ ]:
def make_json(name, seg_ids):
    state = {
        "layers": [
            {
            "source": "graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod",
            "type": "segmentation_with_graph",
            "selectedAlpha": 0.3,
            "segments": seg_ids,
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "graphOperationMarker": [
                {
                "annotations": [],
                "tags": []
                },
                {
                "annotations": [],
                "tags": []
                }
            ],
            "pathFinder": {
                "color": "#ffff00",
                "pathObject": {
                "annotationPath": {
                    "annotations": [],
                    "tags": []
                },
                "hasPath": False
                }
            },
            "name": name
            },
            {
            "type": "segmentation",
            "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
            "objectAlpha": 0.1,
            "hideSegmentZero": False,
            "ignoreSegmentInteractions": True,
            "segmentColors": {
                "1": "#bfbfbf",
                "2": "#d343d6"
            },
            "segments": [
                "1",
                "2"
            ],
            "skeletonRendering": {
                "mode2d": "lines_and_points",
                "mode3d": "lines"
            },
            "name": "neuropil"
            }
        ],
        "navigation": {
            "pose": {
            "position": {
                "voxelSize": [
                4.300000190734863,
                4.300000190734863,
                45
                ],
                "voxelCoordinates": [
                38930,
                138652,
                388
                ]
            }
            },
            "zoomFactor": 2
        },
        "showAxisLines": False,
        "showDefaultAnnotations": False,
        "perspectiveOrientation": [
            0,
            1,
            0,
            0
        ],
        "perspectiveZoom": 7704.152500126457,
        "showSlices": False,
        "selectedLayer": {
            "layer": "neuropil"
        },
        "layout": {
            "type": "3d",
            "orthographicProjection": True
        }
        }

    return state

In [ ]:
# import dataframe
wing_df = pd.read_pickle('preMN_to_MN_wing_v604.pkl')

In [ ]:
# generate a link based on a cell type
# only for local and intersegmental preMNs

idx = pd.IndexSlice
df = wing_df.loc[idx[:,:,:,['local','local_intersegmental'],:]].reset_index() # group by cell type

cell_types = df.cell_type.unique()

for cell_type in cell_types:
    cell_type_label = str(cell_type)
    seg_ids = df[df.cell_type.isin([cell_type_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = cell_type_label,
                   seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}.json'.format(cell_type_label), "w") as outfile:
        json.dump(state, outfile)



In [ ]:
# generate a link based on preferred module

idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()

for module in modules:
    module_label = str(module)
    seg_ids = df[df.preferred_module.isin([module_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = module_label,
                   seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}.json'.format(module_label), "w") as outfile:
        json.dump(state, outfile)


In [803]:
# if you want a link
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5986275473489920'